In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import os
from statsmodels.tsa.statespace.sarimax import SARIMAX
%matplotlib inline

E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#Population values
population = {
    "Chiapas_2016-2017.csv": 5217908,
    "Colima_2016-2017.csv": 711235,
    "Guerrero_2016-2017.csv": 3533251,
    "Hidalgo_2016-2017.csv": 2858359,
    "NuevoLeon_2016-2017.csv": 5119504,
    "Oaxaca_2016-2017.csv": 3967889,
    "QuintanaRoo_2016-2017.csv": 1501562,
    "Tabasco_2016-2017.csv" : 2395272,
    "Veracruz_2016-2017.csv" : 8112505,
    "Yucatan_2016-2017.csv" : 2097175,
    
    "casanare_2016-2017.csv" : 356438,
    "cordoba_2016-2017.csv" : 1709603,
    "cundinamarca_2016-2017.csv" : 2680041,
    "huila_2016-2017.csv" : 1154804,
    "meta_2016-2017.csv" : 961292,
    "santander_2016-2017.csv" : 2061095,
    "santander_norte_2016-2017.csv" : 1355723,
    "tolima_2016-2017.csv" : 1408274,
    "valle_cauca_2016-2017.csv" : 4613377,
    
    "Alagoas_2016-2017.csv": 3375823,
    "Bahia_2016-2017.csv": 15344447,
    "Ceara_2016-2017.csv": 9020460,
    "Goias_2016-2017.csv": 6778772,
    "Maranhao_2016-2017.csv": 7000229,
    "MatoGrosso_2016-2017.csv": 3344544,
    "MinasGerais_2016-2017.csv": 21119536,
    "Para_2016-2017.csv": 8366628,
    "RioDeJaneiro_2016-2017.csv": 16718956,
    "SaoPaulo_2016-2017.csv": 45094866,
}

In [3]:
def formatFilename(filename):
    return filename.replace(".csv", "")

In [5]:
test_index = 52 #Use first year to train, second to test
for (p,d,q) in [(p,d,q) for p in range(4) for d in range(2) for q in range(4)]:
    for (sp, sd, sq, seasonal) in [(sp, sd, sq, seasonal) for sp in range(4)\
                                   for sd in range(2) for sq in range(4) for seasonal in [1,52]]:
        for country in ["Mexico", "Brazil", "Colombia"]:
            print(country)
            print(p,d,q)
            print(sp,sd,sq,seasonal)
            print("---")
            folder = "../../data/{}/processed_data".format(country)
            files = os.listdir(folder)
            for file in files:
                dataset = pd.read_csv("{}/{}".format(folder, file))
                formattedFilename = formatFilename(file)
                #Preprocess data
                dataset["Searches"] /= 100
                state = file
                dataset["Cases"] *= (100000/population[state]) # Transform to incidence per 100,000 habitants    

                #Develop Sarimax model
                model= SARIMAX(dataset["Cases"].values[:test_index],
                            exog=dataset["Searches"].values[:test_index],
                            trend='n', order=(4,0,1), seasonal_order=(0,0,0,52))
                #Fit model
                results = model.fit()
            #     print(results.summary())

                #Predict for next year
                predictions = results.predict(
                start = test_index,
                end= test_index + 51,                       
                dynamic= True,
                exog=dataset[test_index:]["Searches"].values.reshape(52,1))


                #Store results
                predictions_df = pd.DataFrame()
                predictions_df["predictions_incidence"] = predictions
                predictions_df["observed_incidence"] = dataset["Cases"][test_index:].values

                predictions_df["Predictions"] = predictions * (population[state] / 100000)
                predictions_df["Cases"] = dataset["Cases"][test_index:].values * (population[state] / 100000)

                predictions_df["error"] = predictions_df["Cases"] - predictions_df["Predictions"]
                predictions_df.set_index(dataset[52:]["Date"], inplace=True)
                
                outFolder = "{},{},{}-{},{},{},{}".format(p,d,q, sp, sd, sq, seasonal)
                if(not os.path.isdir("{}/{}".format(country, outFolder))):
                    os.mkdir("{}/{}".format(country, outFolder))

                if(not os.path.isdir("{}/{}/{}".format(country, outFolder,file))):
                    os.mkdir("{}/{}/{}".format(country, outFolder, file))

                predictions_df.to_csv("{}/{}/{}/Arimax-{}.csv".format(country, outFolder, file, formattedFilename))

                #Save images
                plt.clf()
                
                cols = ['Cases', 'Predictions']
                colors = ['#2962FF', '#FF9800']

                predictions_df[cols].plot(
                    figsize=(10, 10),
                    grid=True,
                    color=colors
                )
                
                plt.title("Arimax Model\n{}".format(formattedFilename))
                ax = plt.gca()
                ax.set_facecolor((0.9, 0.9, 0.9, 0.7))
                plt.xlabel("Date")
                plt.ylabel("Cases")
                plt.grid(linestyle='dashed', linewidth=1.5)
                fig = plt.gcf()
                fig.savefig("{}/{}/{}/Arimax-{}.png".format(country, outFolder, file, formattedFilename))
                plt.close("all")

Mexico
0 0 0
0 0 0 1
---


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Brazil
0 0 0
0 0 0 1
---


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Colombia
0 0 0
0 0 0 1
---


...

Brazil
0 1 2
3 1 3 1
---


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Colombia
0 1 2
3 1 3 1
---


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


MemoryError: In RendererAgg: Out of memory